In [2]:
import mysql.connector
from functools import partial


def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='Megadados!',
        database='XPTO',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

In [8]:
db("""
SELECT
    pizza, preco - SUM(quantidade * preco_unitario) as lucro
FROM
    Pizza
    INNER JOIN IngredientePizza USING (pizza)
    INNER JOIN Ingrediente USING (ingrediente)
GROUP BY
    pizza
ORDER BY 
    lucro DESC
LIMIT 3
""")

Executando query:
('pão', 95.0)
('incoerente', 83.0)
('legumes', 78.0)


In [18]:
db('DROP TABLE IF EXISTS qtd_pizza')
db("""
CREATE TEMPORARY TABLE qtd_pizza
    SELECT 
        pizza, COUNT(ingrediente) as qtd_ingredientes
    FROM
        IngredientePizza
    GROUP BY 
        pizza
""")
db('''
SELECT * from qtd_pizza
''')

Executando query:
Executando query:
Executando query:
('calabresa', 2)
('incoerente', 4)
('legumes', 2)
('mussarela', 2)
('pão', 1)


In [20]:
db('DROP TABLE IF EXISTS qtd_pizza_tipo')
db("""
CREATE TEMPORARY TABLE qtd_pizza_tipo
    SELECT 
        pizza, COUNT(ingrediente) as qtd_vegano
    FROM
        IngredientePizza
        INNER JOIN IngredienteTipo USING (ingrediente)
    WHERE
        tipo = "vegana"
    GROUP BY 
        pizza
""")
db('''
SELECT * from qtd_pizza_tipo
''')

Executando query:
Executando query:
Executando query:
('legumes', 2)
('incoerente', 2)


In [25]:
db("""
SELECT 
    *
FROM
    qtd_pizza
    INNER JOIN qtd_pizza_tipo USING (pizza)
WHERE
    qtd_ingredientes = qtd_vegano
""")

Executando query:
('legumes', 2, 2)
